# Tarea de Scraping Web con Python y Selenium

## Objetivo
Desarrollar habilidades prácticas en scraping web utilizando Python con Selenium y BeautifulSoup. Se espera que manejen el navegador de forma programática para extraer datos dinámicos de un sitio web específico y que realicen un análisis básico de los datos obtenidos.

## Instrucciones
1. Configura tu entorno de desarrollo instalando las librerías necesarias: Selenium, BeautifulSoup, pandas, entre otras que consideres necesarias.
2. Configura un navegador usando Selenium. Debes asegurarte de incluir opciones como el modo incógnito y el modo sin interfaz gráfica (headless).
3. Elige un sitio web que ofrezca datos dinámicos y sea legal para hacer scraping (por ejemplo, datos meteorológicos, precios de productos, cotizaciones de bolsa).
4. Navega al sitio web utilizando Selenium, realiza búsquedas o filtra datos si es necesario utilizando interacciones del navegador.
5. Extrae datos relevantes usando Selenium y BeautifulSoup. Debes obtener al menos tres tipos de datos relacionados (por ejemplo, nombre del producto, precio y categoría).
6. Limpia y organiza los datos extraídos en un DataFrame de pandas.
7. Realiza un análisis básico de los datos: puede ser estadístico descriptivo o alguna visualización simple.
8. Documenta cada paso del proceso con comentarios en el código y celdas Markdown explicando las decisiones y métodos utilizados.

## Criterios de Evaluación
- Correcta configuración y uso de Selenium y BeautifulSoup.
- Capacidad para navegar y extraer datos de forma efectiva y eficiente.
- Limpieza y estructuración adecuada de los datos extraídos.
- Calidad del análisis realizado y claridad en la documentación.

## Entrega
- Debes entregar este cuaderno Jupyter completado con todo el código, análisis y documentación solicitada.

¡Buena suerte y que disfrutes del proceso de aprendizaje y exploración de datos!


1. Configura tu entorno de desarrollo instalando las librerías necesarias: Selenium, BeautifulSoup, pandas, entre otras que consideres necesarias.

In [98]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import undetected_chromedriver as uc
import time

2. Configura un navegador usando Selenium. Debes asegurarte de incluir opciones como el modo incógnito y el modo sin interfaz gráfica (headless).

In [99]:
options = uc.ChromeOptions()
options.headless = True
options.add_argument("--incognito")
options.add_argument("--disable-infobars")
options.add_argument("--headless") 
driver = uc.Chrome(options=options)

3. Elige un sitio web que ofrezca datos dinámicos y sea legal para hacer scraping (por ejemplo, datos meteorológicos, precios de productos, cotizaciones de bolsa).

In [100]:
home_link = "https://www.ebay.com/"
search_kw = "iphone x".replace(" ","+")

driver.get(home_link+"/sch/i.html?_from=R40&_trksid=p2047675.m570.l1313&_nkw="+search_kw+"&_sacat=0")

4. Navega al sitio web utilizando Selenium, realiza búsquedas o filtra datos si es necesario utilizando interacciones del navegador.

In [101]:
print("Capturando pantalla...")
driver.save_screenshot("screenshot.png")

phone_title = []
phone_link = []
phone_status = []
phone_score = []
phone_reviews_amt = []
phone_price = []
phone_location = []

Capturando pantalla...


5. Extrae datos relevantes usando Selenium y BeautifulSoup. Debes obtener al menos tres tipos de datos relacionados (por ejemplo, nombre del producto, precio y categoría).

In [102]:
pg_amount = 5
page = BeautifulSoup(driver.page_source,'html.parser')

for i in range(0, pg_amount):
  for phone in page.findAll('li', attrs={'class':'s-item', 'data-view':True}):
    title = phone.find('h3', attrs={'class':'s-item__title'})
    if title:
      phone_title.append(title.text)
    else: phone_title.append('')

    link = phone.find('a', attrs={'class':'s-item__link'})
    if link:
      phone_link.append(link['href'])
    else: phone_link.append('')

    status = phone.find('div', attrs={'class':'s-item__subtitle'})
    if status:
      phone_status.append(status.text)
    else: phone_status.append('')

    score = phone.find('div',attrs={'class':['b-starrating','x-star-rating']})
    if score:
      score.find('span', attrs={'class':'clipped'})
      if score:
        phone_score.append(score.text[0:3])
      else: phone_score.append('')
    else: phone_score.append('')

    reviews_amt = phone.find('span',attrs={'class':'s-item__reviews-count'})
    if reviews_amt:
       phone_reviews_amt.append(reviews_amt.span.text[0:reviews_amt.span.text.find('valor')-1])
    else: phone_reviews_amt.append('')

    price = phone.find('span',attrs={'class','s-item__price'})
    if price:
      if ' a ' not in price.text:
        cleaned_price = price.text[5:].replace('\xa0', '')
        phone_price.append(float(cleaned_price))
      else:
        cleaned_price = price.text.split('a')[0]
        cleaned_price = cleaned_price[5:].replace('\xa0', '')
        phone_price.append(float(cleaned_price))
    else: phone_price.append(0)

    location = phone.find('span',attrs={'class':'s-item__location'})
    if location:
      phone_location.append(location.text[3:])
    else: phone_location.append('')


6. Limpia y organiza los datos extraídos en un DataFrame de pandas.

In [103]:
phone_list = pd.DataFrame({                            
                            'STATUS':phone_status,                        
                            'PRICE':phone_price,
                            'LOCATION':phone_location,
                         })
phone_list

,STATUS,PRICE,LOCATION
0,Brand New,0.00,
1,De segunda mano,2458.80,Estados Unidos
2,Restaurados,2325.06,Estados Unidos
3,Restaurados,2441.69,Estados Unidos
4,✅ Envío y devoluciones gratuitos ✅ 60 días de ...,2508.64,Estados Unidos
...,...,...,...
300,Para repuestos solamente,1246.01,Estados Unidos
301,De segunda mano,1869.02,Estados Unidos
302,Para repuestos solamente,1328.91,Estados Unidos
303,Caja abierta,2241.99,Estados Unidos


7. Realiza un análisis básico de los datos: puede ser estadístico descriptivo o alguna visualización simple.

In [113]:
#Comparemos los precios de los teléfonos de segunda mano contra los restaurados
segunda_mano = phone_list[phone_list['STATUS'] == 'De segunda mano']
restaurados = phone_list[phone_list['STATUS'] == 'Restaurados']

print('Precio promedio de los teléfonos de segunda mano:', segunda_mano['PRICE'].mean())
print('Precio promedio de los teléfonos restaurados:', restaurados['PRICE'].mean())

valor_agregado = restaurados['PRICE'].mean() - segunda_mano['PRICE'].mean()
print('Concluimos de aquí que restaurar un iphone 10 tiene un valor agregado de:', valor_agregado, 'pesos')

Precio promedio de los teléfonos de segunda mano: 1862.1424324324323
Precio promedio de los teléfonos restaurados: 2258.8975
Concluimos de aquí que restaurar un iphone 10 tiene un valor agregado de: 396.7550675675677 pesos
